In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# Install Tensorflow from the website: https://www.tensorflow.org/versions/r0.12/get_started/os_setup.html

# Installing Keras
# 


# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [4]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop


from keras import backend as K
from keras import layers
import numpy as np
import tensorflow as tf

class NormalDensity(layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        #self.batch_size = batch_size
        super(NormalDensity, self).__init__(**kwargs)
        
    def build(self, input_shape):
        
        #Define set of traditional weights
        self.w = self.add_weight(name='w', 
                                 shape=(input_shape[1], self.output_dim),
                                 initializer='uniform',
                                 trainable=True)
        
        #Define our plasticity coefficient
        self.alpha = self.add_weight(name='alpha', 
                              shape=(1, 1),
                              initializer='uniform',
                              trainable=True)
        
        
        #The Hebbian trace
        self.hebb = self.add_weight(name='hebb', 
                              shape=(input_shape[1], self.output_dim),
                              initializer='zeros',
                              trainable=False)
        
        #Step size will be optimized
        self.eta = self.add_weight(name='eta', 
                                      shape=(1, 1),
                                      initializer='uniform',
                                      trainable=True)
        super(NormalDensity, self).build(input_shape)

        
        #yout = F.tanh( yin.mm(self.w + torch.mul(self.alpha, hebb)) + input )
        #hebb = (1 - self.eta) * hebb + self.eta * torch.bmm(yin.unsqueeze(2), yout.unsqueeze(1))[0] # bmm here is used to implement an outer product between yin and yout, with the help of unsqueeze (i.e. added empty dimensions)
        #return yout, hebb

    def call(self, x):
        
        #X (layer input)     : shape(?, INPUT_DIM)
        #W                   : shape(INPUT_DIM, OUTPUT_DIM)
        #hebb                : shape(INPUT_DIM, OUTPUT_DIM)
        #Y (layer output)    : shape(?, OUTPUT_DIM)
        #ETA                 : scalar (one per layer)
        
        #yout = K.maximum(0.0, np.add((K.dot(self.y, np.add(K.dot(self.alpha, K.transpose(self.hebb)), self.w))), x))
        #hebb = (1 - 0.01) * self.hebb + 0.01 * K.dot(self.y, yout)
        #yout = K.maximum(0.0, np.add(self.y * np.add(self.alpha * self.hebb, self.w), x))
        
        #y = K.dot(x, self.w)
        #y = F.tanh( yin.mm(self.w + torch.mul(self.alpha, hebb)) + yin )
        #plastic_y = self.alpha * (K.dot(x, self.hebb)) 
        #self.hebb = (1 - self.eta) * self.hebb + self.eta * torch.bmm(yin.unsqueeze(2), yout.unsqueeze(1))[0]
        #model_out = K.maximum(0.0, y + plastic_y)
        
        #Hebbian update - option 1
        #self.hebb = self.eta * K.dot(x, model_out) + (1 - self.eta) * self.hebb
        #print(self.hebb)
        
        #Hebbian update - option 2
        #self.hebb +=self.eta * K.dot(model_out, (x - (K.dot(model_out, self.hebb))))
        
        y = K.dot(x, self.w)
        plastic_y = self.alpha * (K.dot(x, self.hebb))   
        model_out = K.maximum(0.0, y + plastic_y)
        
        #Hebbian update - option 1
        self.hebb = self.eta * K.dot(x, model_out) + (1 - self.eta) * self.hebb
        print(self.hebb)

        return model_out
        #return K.maximum(0.0, y)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


In [8]:
# Initialising the ANN
classifier = Sequential()

In [9]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))
#classifier.add(NormalDensity(6, input_shape=(11,)))


/Users/yunuskocyigit/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
  


In [10]:
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))

/Users/yunuskocyigit/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  


In [11]:
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

/Users/yunuskocyigit/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [12]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [13]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

/Users/yunuskocyigit/anaconda/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
8000/8000 [==============================] - 1s 129us/step - loss: 0.4822 - acc: 0.7959
Epoch 2/100
8000/8000 [==============================] - 1s 108us/step - loss: 0.4256 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4194 - acc: 0.8069
Epoch 4/100
8000/8000 [==============================] - 1s 119us/step - loss: 0.4165 - acc: 0.8274
Epoch 5/100
8000/8000 [==============================] - 1s 118us/step - loss: 0.4145 - acc: 0.8296
Epoch 6/100
8000/8000 [==============================] - 1s 120us/step - loss: 0.4135 - acc: 0.8310
Epoch 7/100
8000/8000 [==============================] - 1s 108us/step - loss: 0.4120 - acc: 0.8311
Epoch 8/100
8000/8000 [==============================] - 1s 110us/step - loss: 0.4110 - acc: 0.8327
Epoch 9/100
8000/8000 [==============================] - 1s 109us/step - loss: 0.4105 - acc: 0.8322
Epoch 10/100
8000/8000 [==============================] - 1s 110us/step - loss: 0.4093 - acc: 0.8332

8000/8000 [==============================] - 1s 112us/step - loss: 0.3957 - acc: 0.8347
Epoch 83/100
8000/8000 [==============================] - 1s 108us/step - loss: 0.3959 - acc: 0.8366
Epoch 84/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.3959 - acc: 0.8362
Epoch 85/100
8000/8000 [==============================] - 1s 109us/step - loss: 0.3954 - acc: 0.8386
Epoch 86/100
8000/8000 [==============================] - 1s 110us/step - loss: 0.3950 - acc: 0.8364
Epoch 87/100
8000/8000 [==============================] - 1s 111us/step - loss: 0.3956 - acc: 0.8371
Epoch 88/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.3952 - acc: 0.8370
Epoch 89/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.3948 - acc: 0.8365
Epoch 90/100
8000/8000 [==============================] - 1s 107us/step - loss: 0.3953 - acc: 0.8354
Epoch 91/100
8000/8000 [==============================] - 1s 107us/step - loss: 0.3947 - acc: 0.8370
Epo

In [13]:
# Part 3 - Making the predictions and evaluating the model
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [14]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)